In [105]:
import os
import pandas as pd

# Set the directory where the text files are located
songs_dir = "songs"
data = []
# Loop through each file in the directory
for root, dirs, files in os.walk(songs_dir):
    for file in files:
        if file.endswith(".txt"):
            artist = os.path.basename(root)
            with open(os.path.join(root, file), 'r', encoding="utf8") as f:
                lyrics = f.read().replace('\n', ' ')
                # Add the data to the DataFrame
                data.append([artist, lyrics])
                
# Create an empty DataFrame to store the data
df = pd.DataFrame(data, columns=['Artist', 'Lyrics'])
# Export the DataFrame to a CSV file
df.to_csv('lyrics.csv', index=False)


In [106]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Load the data from the CSV file
data = pd.read_csv("lyrics.csv")

nltk.download('stopwords')
nltk.download('punkt')

# Create a list of stopwords to remove
stop_words = set(stopwords.words("english"))
stop_words.add("verse")
stop_words.add("intro")

# Create a stemmer to use for word stemming
stemmer = PorterStemmer()

# Preprocess each lyric in the DataFrame
for i, row in data.iterrows():
#Convert the lyric to lowercase
    lyric = str(row["Lyrics"]).lower()
    match = re.search(r'lyrics\[[^\]]*\]', lyric)

    # Check if the split was successful
    if match:
        split_index = match.end()
        cleaned_lyric = lyric[split_index:]
    else:
        cleaned_lyric = lyric

    #Tokenize the lyric into words
    words = word_tokenize(cleaned_lyric)

    #Remove stop words and punctuation
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]

    #Stem each word
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    #Join the stemmed words back into a single string
    preprocessed_lyric = " ".join(stemmed_words)

    #Replace the original lyric with the preprocessed lyric in the DataFrame
    data.at[i, "Lyrics"] = preprocessed_lyric

# Export the preprocessed DataFrame to a CSV file
data.to_csv("preprocessed_lyrics.csv", index=False)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jarraomar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jarraomar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

pre_processed_data = pd.read_csv("preprocessed_lyrics.csv")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pre_processed_data["Lyrics"])
sequences = tokenizer.texts_to_sequences(pre_processed_data["Lyrics"])

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#For Artist Classification

artists = pre_processed_data["Artist"].unique()
# Encode the artist names as integer labels
label_encoder = LabelEncoder()
pre_processed_data["Artist"] = label_encoder.fit_transform(pre_processed_data["Artist"])

# Initialize empty dataframes for training, validation, and testing
train_df = pd.DataFrame(columns=["Artist", "Lyrics"])
val_df = pd.DataFrame(columns=["Artist", "Lyrics"])
test_df = pd.DataFrame(columns=["Artist", "Lyrics"])

for artist in artists:
    # Get the data for the current artist
    artist_data = pre_processed_data[pre_processed_data["Artist"] == artist]
    
    # Split the artist data into training, validation, and testing sets
    artist_train, artist_test = train_test_split(artist_data, test_size=0.2, random_state=42)
    artist_train, artist_val = train_test_split(artist_train, test_size=0.2, random_state=42)
    
    # Concatenate the artist training, validation, and testing dataframes with the overall training, validation, and testing dataframes
    train_df = pd.concat([train_df, artist_train])
    val_df = pd.concat([val_df, artist_val])
    test_df = pd.concat([test_df, artist_test])

# Create a directory to store the CSV files
directory = "data_splits"
if not os.path.exists(directory):
    os.makedirs(directory)
    train_df.to_csv(os.path.join(directory, "train.csv"), index=False)
    val_df.to_csv(os.path.join(directory, "val.csv"), index=False)
    test_df.to_csv(os.path.join(directory, "test.csv"), index=False)

train_df.to_csv(os.path.join(directory, "train.csv"), index=False)
val_df.to_csv(os.path.join(directory, "val.csv"), index=False)
test_df.to_csv(os.path.join(directory, "test.csv"), index=False)


In [110]:
import pandas as pd
import numpy as np
import gensim
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

train = pd.read_csv('data_splits/train.csv')
val = pd.read_csv('data_splits/val.csv')
test = pd.read_csv('data_splits/test.csv')

#Train a Word2Vec model on the preprocessed lyrics data:
# sentences = [row.split() for row in train['Lyrics']]
# model = gensim.models.Word2Vec(sentences, min_count=1)

#Creation of MLP model:
model_mlp = Sequential()
model_mlp.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, input_length=200))
model_mlp.add(GlobalMaxPooling1D())
model_mlp.add(Dense(100, activation='relu'))
model_mlp.add(Dropout(0.4))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(8, activation='softmax'))

tokenizer.fit_on_texts(train['Lyrics'])
sequences_train = tokenizer.texts_to_sequences(train['Lyrics'])
sequences_val = tokenizer.texts_to_sequences(val['Lyrics'])
sequences_test = tokenizer.texts_to_sequences(test['Lyrics'])
word_index = tokenizer.word_index
X_train = pad_sequences(sequences_train, maxlen=200)
X_val = pad_sequences(sequences_val, maxlen=200)
X_test = pad_sequences(sequences_test, maxlen=200)

# One-hot encode the target variable
y_train = to_categorical(train['Artist'])
y_val = to_categorical(val['Artist'])
y_test = to_categorical(test['Artist'])


In [111]:
#We apply earlystopping in order to avoid over-fitting
es = EarlyStopping(monitor='val_loss', patience=5)
model_mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [112]:
model_mlp.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64, callbacks=[es])

Epoch 1/100
4/4 [==============================] - 2s 112ms/step - loss: 2.0842 - accuracy: 0.1328 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 2/100
4/4 [==============================] - 0s 62ms/step - loss: 2.0795 - accuracy: 0.1367 - val_loss: 2.0793 - val_accuracy: 0.1250
Epoch 3/100
4/4 [==============================] - 0s 55ms/step - loss: 2.0799 - accuracy: 0.1094 - val_loss: 2.0791 - val_accuracy: 0.1250
Epoch 4/100
4/4 [==============================] - 0s 50ms/step - loss: 2.0806 - accuracy: 0.0938 - val_loss: 2.0788 - val_accuracy: 0.1250
Epoch 5/100
4/4 [==============================] - 0s 52ms/step - loss: 2.0834 - accuracy: 0.1406 - val_loss: 2.0782 - val_accuracy: 0.1250
Epoch 6/100
4/4 [==============================] - 0s 51ms/step - loss: 2.0867 - accuracy: 0.1133 - val_loss: 2.0778 - val_accuracy: 0.1250
Epoch 7/100
4/4 [==============================] - 0s 51ms/step - loss: 2.0776 - accuracy: 0.1602 - val_loss: 2.0777 - val_accuracy: 0.1250
Epoch 8/100
4/4 [==

In [113]:
#Testing our model's accuracy based on a separate test set. "X and Y test"
test_loss, test_acc = model_mlp.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

3/3 - 0s - loss: 0.7355 - accuracy: 0.7625 - 153ms/epoch - 51ms/step
Test accuracy: 0.762499988079071


In [104]:
from keras import backend as K
K.clear_session()
del model_mlp